In [ ]:
# This is a slightly adapted version of the ALT code, which replicates the example at the bottom of DR.py

In [1]:
import numpy as np
from numpy import *
import pandas as pd
from scipy import stats
from itertools import groupby


In [2]:
def Teff2bc2lum(teff, plx, vmag, Av=0):
    
    lteff = np.log10(teff)
    
    BCv = np.full(len(lteff), -100.5)

    idx0 = lteff<3.70
    p0 = [-0.190537291496456*10.0**5,
           0.155144866764412*10.0**5,
          -0.421278819301717*10.0**4,
           0.381476328422343*10.0**3,]
    BCv[idx0] = polyval(p0[::-1], lteff[idx0])

    idx1 = (3.70<lteff) & (lteff<3.90)
    p1 = [-0.370510203809015*10.0**5,
           0.385672629965804*10.0**5,
          -0.150651486316025*10.0**5,
           0.261724637119416*10.0**4,
          -0.170623810323864*10.0**3,]
    BCv[idx1] = polyval(p1[::-1], lteff[idx1])

    idx2 = lteff>3.90
    p2 = [-0.118115450538963*10.0**6,
                 0.137145973583929*10.0**6,
                -0.636233812100225*10.0**5,
                 0.147412923562646*10.0**5,
                -0.170587278406872*10.0**4,
                 0.788731721804990*10.0**2,]
    BCv[idx2] = polyval(p2[::-1], lteff[idx2])

    lum = 10**(4.0+0.4*4.73-2.0*np.log10(plx)-0.4*(vmag - Av + BCv)) # in solar units
    
    return lum

In [3]:
def bv2teff(b_v):
    p = [-0.359495739295671,
           2.192970376522490,
          -5.396909891322525,
           6.792599779944473,
          -4.608815154057166,
           1.740690042385095,
          -0.654992268598245,
           3.979145106714099]

    return 10.0**np.polyval(p,b_v)

In [4]:
def add_V_I(whole, cond, p):
    x = whole['B-V'][cond] - 1
    whole['V-I'][cond] = polyval(p[::-1], x) + 1
    

def BV2VI(bv, vmag, g_mag_abs):

    whole = pd.DataFrame(data={'B-V': bv, 'Vmag': vmag, 'g_mag_abs': g_mag_abs})

    # Mg: empirical relation from Tiago to separate dwarfs from giants
    # note: this relation is observational; it was made with REDDENED B-V and g_mag values
    Mg = 6.5*whole['B-V'] - 1.8

    # B-V-to-teff limits from (6) fig 5
    whole = whole[(whole['B-V'] > -0.4) & (whole['B-V'] < 1.7)]

    # B-V limits for dwarfs and giants, B-V conditions from (1)
    # if a star can't be classified as dwarf or giant, remove it
    condG  = (whole['B-V'] > -0.25) & (whole['B-V'] < 1.75) & (Mg > whole['g_mag_abs'])
    condD1 = (whole['B-V'] > -0.23) & (whole['B-V'] < 1.4)  & (Mg < whole['g_mag_abs'])
    condD2 = (whole['B-V'] > 1.4)   & (whole['B-V'] < 1.9)  & (Mg < whole['g_mag_abs'])

    whole = pd.concat([whole[condG], whole[condD1], whole[condD2]], axis=0)
    
    whole['V-I'] = 100. # write over these values for dwarfs and giants separately

    # coefficients for giants and dwarfs
    cg = [-0.8879586e-2, 0.7390707, 0.3271480, 0.1140169e1, 
          -0.1908637, -0.7898824, 0.5190744, 0.5358868]
    cd1 = [0.8906590e-1, 0.1319675e1, 0.4461807, -0.1188127e1, 
           0.2465572, 0.8478627e1, 0.1046599e2, 0.3641226e1]
    cd2 = [-0.5421588e2, 0.8011383e3, -0.4895392e4, 0.1628078e5, -0.3229692e5,
            0.3939183e5, -0.2901167e5, 0.1185134e5, -0.2063725e4]

    add_V_I(whole, condG, cg)
    add_V_I(whole, condD1, cd1)
    add_V_I(whole, condD2, cd2)
  
    # calculate Imag from V-I and reredden it
    whole['Imag'] = whole['Vmag'] - whole['V-I']
    
    return whole['B-V'], whole['Vmag'], whole['g_mag_abs'], whole['V-I'], whole['Imag']

In [5]:
def seismicParameters(teff, lum):

    # solar parameters
    teff_solar = 5777.0 # Kelvin
    teffred_solar = 8907.0 #Kelvin
    numax_solar = 3090.0 # muHz
    dnu_solar = 135.1 # muHz

 
    
    teffred = teffred_solar*(lum**-0.093) # from (6) eqn 8. red-edge temp
    rad = lum**0.5 * ((teff/teff_solar)**-2) # Steffan-Boltzmann law
    
    numax = numax_solar*(rad**-1.85)*((teff/teff_solar)**0.92) # from (14)

    return rad, numax, teffred, teff_solar, teffred_solar, numax_solar, dnu_solar

In [6]:
def pixel_cost(x, mask_size='new'):
    """ The number of pixels in the aperture. 
    
    Use when calculating instrumental noise with TESS in calc_noise().

    Kewargs
    mask_size ('conservative' or 'new'): the mask size to use for the pixel cost.
    """

    if mask_size == 'conservative':
        N = np.ceil(10.0**-5.0 * 10.0**(0.4*(20.0-x)))
        npix_aper = 10*(N+10)

    if mask_size == 'new':
        # updated number of pixels equation from Tiago on 22.03.18
        npix_aper = np.ceil(10**(0.8464 - 0.2144 * (x - 10.0)))

    total = np.cumsum(npix_aper)
    per_cam = 26*4 # to get from the total pixel cost to the cost per camera at a given time, divide by this
    pix_limit = 1.4e6 # the pixel limit per camera at a given time

    return npix_aper

In [9]:
def granulation(nu0, dilution, a_nomass, b1, b2, vnyq):

    # Divide by dilution squared as it affects stars in the time series.
    # The units of dilution change from ppm to ppm^2 microHz^-1 when going from the
    # time series to frequency. p6: c=4 and zeta = 2*sqrt(2)/pi
    
    Pgran = 2*np.sqrt(2)/np.pi * a_nomass**2 / dilution**2 * (1/b1 / (1 + (nu0/b1)**4) + 1/b2 / (1 + (nu0/b2)**4)) 

    # From (9). the amplitude suppression factor. Normalised sinc with pi (area=1)
    eta = np.sinc(nu0/(2*vnyq))

    # the granulation after attenuation
    Pgran = Pgran * eta**2

    return Pgran, eta

In [8]:
def calc_noise(imag, exptime, teff, e_lng = 0, e_lat = 30, g_lng = 96, g_lat = -30, 
               subexptime = 2.0, npix_aper = 4, frac_aper = 0.76, e_pix_ro = 10, 
               geom_area = 60.0, pix_scale = 21.1, sys_limit = 0):

    omega_pix = pix_scale**2.0
    n_exposures = exptime/subexptime

    # electrons from the star
    megaph_s_cm2_0mag = 1.6301336 + 0.14733937*(teff/5000 - 1)
    e_star = 10.0**(-0.4*imag) * 1e6 * megaph_s_cm2_0mag * geom_area * exptime * frac_aper
    e_star_sub = e_star*subexptime/exptime

    # e/pix from zodi
    dlat = abs(e_lat)/90-1
    vmag_zodi = 23.345 - 1.148*dlat**2
    e_pix_zodi = 10**(-0.4*(vmag_zodi-22.8)) * 2.39e-3 * geom_area * omega_pix * exptime

    # e/pix from background stars
    dlat = abs(g_lat)/40

    dlon = g_lng
    q = np.where(dlon>180.0)
    if len(q[0])>0:
        dlon[q] = 360.0-dlon[q]

    dlon = abs(dlon)/180
    p = [18.97338, 8.833, 4.007, 0.805]
    imag_bgstars = p[0] + p[1]*dlat + p[2]*dlon**p[3]
    e_pix_bgstars = 10**(-0.4*imag_bgstars) * 1.7e6 * geom_area * omega_pix * exptime

    # compute noise sources
    noise_star = np.sqrt(e_star) / e_star
    noise_sky  = np.sqrt(npix_aper*(e_pix_zodi + e_pix_bgstars)) / e_star
    noise_ro   = np.sqrt(npix_aper*n_exposures)*e_pix_ro / e_star
    noise_sys  = 0.0*noise_star + sys_limit/1e6/np.sqrt(exptime/3600)

    noise1 = np.sqrt(noise_star**2 + noise_sky**2 + noise_ro**2)
    noise2 = np.sqrt(noise_star**2 + noise_sky**2 + noise_ro**2 + noise_sys**2)

    return noise2

In [16]:
def globalDetections(g_lng, g_lat, e_lng, e_lat, imag, lum, rad, teff, numax, Nq, teffred, teff_solar, 
                     teffred_solar, numax_solar, dnu_solar, sys_limit, dilution, vary_beta=False, fap = 0.05):

    dnu = dnu_solar*(rad**-1.42)*((teff/teff_solar)**0.71) # from (14) eqn 21
    
    beta = 1.0-np.exp(-(teffred-teff)/1550) # beta correction for hot solar-like stars from (6) eqn 9.
    
    if isinstance(teff, float):  # for only 1 star
        if (teff>=teffred):
            beta = 0.0
    else:
        beta[teff>=teffred] = 0.0

    # to remove the beta correction, set Beta=1
    if vary_beta == False:
        beta = 1.0

    # modified from (6) eqn 11. Now consistent with dnu proportional to numax^0.77 in (14)
    amp = 0.85*2.5*beta*(rad**1.85)*((teff/teff_solar)**0.57)

    # From (5) table 2 values for delta nu_{env}. env_width is defined as +/- some value.
    env_width = 0.66 * numax**0.88
    env_width[numax>100.] = numax[numax>100.]/2.  # from (6) p12

    
    # TESS SPECIFIC
    #total, per_cam, pix_limit, npix_aper = pixel_cost(imag)  # using the old function in this script
    
    cadence = 120 # PLATO cadence (s)
    vnyq = (1.0 / (2.0*cadence)) * 10**6 # in micro Hz
    
    npix_aper = pixel_cost(imag)

    noise = calc_noise(imag=imag, teff=teff, exptime=cadence, g_lng=g_lng, 
                       g_lat=g_lat, sys_limit=sys_limit, npix_aper=npix_aper)
    
    noise *= 1e6 # total noise in units of ppm

    a_nomass = 0.85 * 3382*numax**-0.609 # multiply by 0.85 to convert to redder TESS bandpass.
    b1 = 0.317 * numax**0.970
    b2 = 0.948 * numax**0.992

    # call the function for the real and aliased components (above and below vnyq) of the granulation
    # the order of the stars is different for the aliases so fun the function in a loop
    Pgran, eta = granulation(numax, dilution, a_nomass, b1, b2, vnyq)
    Pgranalias = np.zeros(len(Pgran))
    etaalias = np.zeros(len(eta))

    # if vnyq is 1 fixed value
    
    if isinstance(vnyq, float):
        for i in range(len(numax)):

            if numax[i] > vnyq:
                Pgranalias[i], etaalias[i] = granulation((vnyq - (numax[i] - vnyq)), dilution, a_nomass[i], 
                                                         b1[i], b2[i], vnyq)

            elif numax[i] < vnyq:
                Pgranalias[i], etaalias[i] = granulation((vnyq + (vnyq - numax[i])), dilution, a_nomass[i], 
                                                         b1[i], b2[i], vnyq)

    # if vnyq varies for each star
    else:
        for i in range(len(numax)):

            if numax[i] > vnyq[i]:
                Pgranalias[i], etaalias[i] = granulation((vnyq[i] - (numax[i] - vnyq[i])), dilution, a_nomass[i],
                                                         b1[i], b2[i], vnyq[i])

            elif numax[i] < vnyq[i]:
                Pgranalias[i], etaalias[i] = granulation((vnyq[i] + (vnyq[i] - numax[i])), dilution, a_nomass[i],
                                                         b1[i], b2[i], vnyq[i])

    Pgrantotal = Pgran + Pgranalias

    ptot = 0.5*2.94*amp**2 * 2*env_width/dnu*eta**2. / dilution**2
    Binstr = 2*noise**2 * cadence*1e-6 # from (6) eqn 18
    bgtot = (Binstr + Pgrantotal) * 2*env_width # units are ppm**2

    snr = ptot/bgtot # global signal to noise ratio from (11)
    
    pfinal = np.full(rad.shape[0], -99)
    idx = Nq > 0 # stars where Nq is > 0
    
    Ndpq = 90 # Number of days per observing quarter 
    tlen = Nq[idx]*Ndpq*86400.0 # Length of observations in seconds

    bin_width= 1e6 / tlen

    nbins=(2*env_width[idx]/bin_width).astype(int) # from (11)
    
    snrthresh = stats.chi2.ppf(1.0 - fap, 2*nbins) / (2.0*nbins) - 1

    pfinal[idx] = stats.chi2.sf((snrthresh+1) / (snr[idx]+1)*2*nbins, 2*nbins)
    
    return pfinal, snr # snr is needed in TESS_telecon2.py

In [17]:
# parallax
plx = np.array([30., 20.])

# Photometry
vmag = np.array([5., 6.])
bv = np.array([0.5, 0.7])
g_mag_abs = np.array([5., 7.])

# Galactic coords
g_lng = np.array([96., 20.])
g_lat = np.array([-30., 50.])

# Ecliptic coords
e_lng = np.array([0., 10.])
e_lat = np.array([30., 70.])

teff = bv2teff(bv)

lum = Teff2bc2lum(teff, plx, vmag)

bv, vmag, g_mag_abs, vi, imag = BV2VI(bv, vmag, g_mag_abs)

rad, numax, teffred, teff_solar, teffred_solar, numax_solar, dnu_solar = seismicParameters(teff=teff, lum=lum)

Nq = np.array([1, 1])

pfinal, snr = globalDetections(g_lng=g_lng, g_lat=g_lat, e_lng=e_lng, e_lat=g_lat, imag=imag, lum=lum, rad=rad, 
                               teff=teff, numax=numax, Nq=Nq, teffred=teffred, teff_solar=teff_solar, 
                               teffred_solar=teffred_solar, numax_solar=numax_solar, dnu_solar=dnu_solar, 
                               sys_limit=0., dilution=1., vary_beta=True)
print(pfinal)
print(snr.values)

[0 1]
[0.32744408 0.73003381]


In [ ]:
fieldname l b Gc logAge M_H m_ini mu0 Av mratio Mass logL logTe logg label U B V R I J H Ks Hp_Hipp BT_Tycho VT_Tycho W_Plato crowd